In [ ]:
from pathlib import Path
import numpy as np
import numpy.typing as npt
import h5py
import matplotlib.pyplot as plt
from topostats.io import LoadScans
from topostats.plotting import Colormap

colormap = Colormap()
cmap = colormap.get_cmap()
vmin = -3
vmax = 4
square_markersize_figsize_20 = 3.8

In [ ]:
base_dir = Path("/Users/sylvi/topo_data/topostats_2/datasets/20250528_RA_dose_0GY_picoz")
data_dir = base_dir / "output_fig_nodestats"
figure_dir = Path("/Users/sylvi/topo_data/topostats_2/figures/fig-disordered-tracing")
assert figure_dir.exists()
assert data_dir.exists()

image_filename = "20250528_picoz_0RA_tip_0.0_00019"

image_topostats_file_path = data_dir / "processed" / (f"{image_filename}.topostats")
assert image_topostats_file_path.exists()

loadscans = LoadScans(img_paths=[image_topostats_file_path], channel="dummy")
loadscans.get_data()
loadscans_dict = loadscans.img_dict

image_dict = loadscans_dict[image_filename]
print(image_dict.keys())
image = image_dict["image"]
p2nm = image_dict["pixel_to_nm_scaling"]
print(f"pixel to nm scaling: {p2nm} nm/pixel")
print(f"image size: {image.shape} px, {image.shape[0]*p2nm:.2f} x {image.shape[1]*p2nm:.2f} nm")

image_disordered_traces = image_dict["disordered_traces"]["above"]
image_nodestats = image_dict["nodestats"]["above"]
image_ordered_tracing = image_dict["ordered_traces"]["above"]

image_nodestats_images_grain1 = image_nodestats["images"]["grain_1"]
image_nodestats_images_grain1_grain = image_nodestats_images_grain1["grain"]
image_nodestats_images_grain1_grain_image = image_nodestats_images_grain1_grain["grain_image"]

image_disordered_traces_grain1 = image_disordered_traces["grain_1"]
image_disordered_traces_grain1_branch_indexes = image_disordered_traces_grain1["branch_indexes"]




In [ ]:
# colouring by branch index
# create a mask the the branch indexes
image_disordered_traces_grain1_branch_indexes_mask = np.ma.masked_where(
    image_disordered_traces_grain1_branch_indexes == 0,
    image_disordered_traces_grain1_branch_indexes,
)

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(image_nodestats_images_grain1_grain_image, cmap=cmap, vmin=vmin, vmax=vmax)
ax.imshow(image_disordered_traces_grain1_branch_indexes_mask, cmap="Accent", alpha=1)
plt.axis("off")
plt.savefig(figure_dir / f"i19-fig-ordered-tracing-branch-indexes-grain-1.png", bbox_inches="tight")


In [ ]:
# colouring by molecule
mol_0_ordered_coords = image_ordered_tracing["grain_1"]["mol_0"]["ordered_coords"]
